In [2]:
import torch
import gym
from gym import envs
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
from torch.autograd import Variable
from torch.distributions import Categorical
from torchvision import transforms
from torch.utils.tensorboard import SummaryWriter

In [3]:
lenobs = 100800
class ActorCritic(nn.Module):
    def __init__(self):
        super(ActorCritic, self).__init__()
        self.l1 = nn.Linear(lenobs,25)
        self.l2 = nn.Linear(25,50)
        self.actor_lin1 = nn.Linear(50,2)
        self.l3 = nn.Linear(50,25)
        self.critic_lin1 = nn.Linear(25,1)

    def forward(self,x):
        x = F.normalize(x,dim=0)
        y = F.relu(self.l1(x))
        y = F.normalize(y,dim=0)
        y = F.relu(self.l2(y))
        y = F.normalize(y,dim=0)
#         actor = F.softmax(self.actor_lin1(y),dim=0)
        actor = F.log_softmax(self.actor_lin1(y),dim=0)
        c = F.relu(self.l3(y.detach()))
        critic = torch.tanh(self.critic_lin1(c))
        return actor, critic

In [4]:
tb = SummaryWriter()
env = gym.make('PongNoFrameskip-v0')
moveMapping = {
    0:2,
    1:3
}

model = ActorCritic()
optimizer = optim.Adam(lr=1e-4,params=model.parameters())

model.train()
'''
loss = Variable(loss, requires_grad = True)
actor_loss = Variable(actor_loss, requires_grad = True)
critic_loss = Variable(critic_loss, requires_grad = True)
'''

for i_episode in range(20000):
    print('Epoch {}'.format(i_episode))
#     reward = 0.0
    values = []
    rewards = []
    logprobs = []
    observation = env.reset()
    print('---------------')
    done = False
    N = 0
    while done == False and N<10:
        N+=1
#         print(t)
        pobservation = torch.from_numpy(observation)
        flattened_pobservation = pobservation.view(-1).float()
        policy, value = model(flattened_pobservation)
#         print('Policy:{}'.format(policy))
        values.append(value.item())
#         print('Values:{}'.format(values))
        sampler = Categorical(policy)
        action = sampler.sample()
#         print(action.item())
#         action = np.random.choice(np.array([0,1]), p = policy.view(2,).data.numpy())
        logprobs.append(policy[action.item()].item())
#         print('Logprobs: {}'.format(logprobs))
#         print('Action: {}'.format('right' if action==2 else 'left'))
        observation, reward, done, log = env.step(moveMapping[action.item()])
#         rewards.append(reward)
#         print('rewards: {}'.format(rewards))
#         print('---')
        if done:
            rewards.append(-1.0)
        else:
            rewards.append(1.0)
        
      
    # Reversing because earlier actions need to be discounted
    torch_values = torch.tensor(values, requires_grad = True).view(-1).flip(0)
    torch_rewards = torch.tensor(rewards, requires_grad = True).flip(0)
    torch_logprobs = torch.tensor(logprobs, requires_grad = True).flip(0)
#     print((torch_logprobs<0).sum()/torch_logprobs.shape[0])
    

    
    
    returns = []
    gamma = 0.90
    clc = 0.1
    ret = torch.tensor([0])
    for r in torch_rewards:
        ret = r + gamma*ret
        returns.append(ret)
    returns = torch.tensor(returns, requires_grad = True)
#     print(returns.shape)
    returns = F.normalize(returns,dim=0)
#     print('returns.mean():{} returns.std():{}'.format(returns.mean(), returns.std()))
    actor_loss = -1*torch_logprobs * (returns - torch_values.detach())
    critic_loss = torch.pow(torch_values - returns,2)
    loss = actor_loss.sum() + clc*critic_loss.sum()
    tb.add_scalar('Loss',loss,i_episode)
#     print('Loss: {}'.format(loss))
#     print('Starting Backpropagation')
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

print('Completed Training')

Epoch 0
---------------
Epoch 1
---------------
Epoch 2
---------------
Epoch 3
---------------
Epoch 4
---------------
Epoch 5
---------------
Epoch 6
---------------
Epoch 7
---------------
Epoch 8
---------------
Epoch 9
---------------
Epoch 10
---------------
Epoch 11
---------------
Epoch 12
---------------
Epoch 13
---------------
Epoch 14
---------------
Epoch 15
---------------
Epoch 16
---------------
Epoch 17
---------------
Epoch 18
---------------
Epoch 19
---------------
Epoch 20
---------------
Epoch 21
---------------
Epoch 22
---------------
Epoch 23
---------------
Epoch 24
---------------
Epoch 25
---------------
Epoch 26
---------------
Epoch 27
---------------
Epoch 28
---------------
Epoch 29
---------------
Epoch 30
---------------
Epoch 31
---------------
Epoch 32
---------------
Epoch 33
---------------
Epoch 34
---------------
Epoch 35
---------------
Epoch 36
---------------
Epoch 37
---------------
Epoch 38
---------------
Epoch 39
---------------
Epoch 40
-

Epoch 325
---------------
Epoch 326
---------------
Epoch 327
---------------
Epoch 328
---------------
Epoch 329
---------------
Epoch 330
---------------
Epoch 331
---------------
Epoch 332
---------------
Epoch 333
---------------
Epoch 334
---------------
Epoch 335
---------------
Epoch 336
---------------
Epoch 337
---------------
Epoch 338
---------------
Epoch 339
---------------
Epoch 340
---------------
Epoch 341
---------------
Epoch 342
---------------
Epoch 343
---------------
Epoch 344
---------------
Epoch 345
---------------
Epoch 346
---------------
Epoch 347
---------------
Epoch 348
---------------
Epoch 349
---------------
Epoch 350
---------------
Epoch 351
---------------
Epoch 352
---------------
Epoch 353
---------------
Epoch 354
---------------
Epoch 355
---------------
Epoch 356
---------------
Epoch 357
---------------
Epoch 358
---------------
Epoch 359
---------------
Epoch 360
---------------
Epoch 361
---------------
Epoch 362
---------------
Epoch 363
--

Epoch 641
---------------
Epoch 642
---------------
Epoch 643
---------------
Epoch 644
---------------
Epoch 645
---------------
Epoch 646
---------------
Epoch 647
---------------
Epoch 648
---------------
Epoch 649
---------------
Epoch 650
---------------
Epoch 651
---------------
Epoch 652
---------------
Epoch 653
---------------
Epoch 654
---------------
Epoch 655
---------------
Epoch 656
---------------
Epoch 657
---------------
Epoch 658
---------------
Epoch 659
---------------
Epoch 660
---------------
Epoch 661
---------------
Epoch 662
---------------
Epoch 663
---------------
Epoch 664
---------------
Epoch 665
---------------
Epoch 666
---------------
Epoch 667
---------------
Epoch 668
---------------
Epoch 669
---------------
Epoch 670
---------------
Epoch 671
---------------
Epoch 672
---------------
Epoch 673
---------------
Epoch 674
---------------
Epoch 675
---------------
Epoch 676
---------------
Epoch 677
---------------
Epoch 678
---------------
Epoch 679
--

Epoch 959
---------------
Epoch 960
---------------
Epoch 961
---------------
Epoch 962
---------------
Epoch 963
---------------
Epoch 964
---------------
Epoch 965
---------------
Epoch 966
---------------
Epoch 967
---------------
Epoch 968
---------------
Epoch 969
---------------
Epoch 970
---------------
Epoch 971
---------------
Epoch 972
---------------
Epoch 973
---------------
Epoch 974
---------------
Epoch 975
---------------
Epoch 976
---------------
Epoch 977
---------------
Epoch 978
---------------
Epoch 979
---------------
Epoch 980
---------------
Epoch 981
---------------
Epoch 982
---------------
Epoch 983
---------------
Epoch 984
---------------
Epoch 985
---------------
Epoch 986
---------------
Epoch 987
---------------
Epoch 988
---------------
Epoch 989
---------------
Epoch 990
---------------
Epoch 991
---------------
Epoch 992
---------------
Epoch 993
---------------
Epoch 994
---------------
Epoch 995
---------------
Epoch 996
---------------
Epoch 997
--

---------------
Epoch 1269
---------------
Epoch 1270
---------------
Epoch 1271
---------------
Epoch 1272
---------------
Epoch 1273
---------------
Epoch 1274
---------------
Epoch 1275
---------------
Epoch 1276
---------------
Epoch 1277
---------------
Epoch 1278
---------------
Epoch 1279
---------------
Epoch 1280
---------------
Epoch 1281
---------------
Epoch 1282
---------------
Epoch 1283
---------------
Epoch 1284
---------------
Epoch 1285
---------------
Epoch 1286
---------------
Epoch 1287
---------------
Epoch 1288
---------------
Epoch 1289
---------------
Epoch 1290
---------------
Epoch 1291
---------------
Epoch 1292
---------------
Epoch 1293
---------------
Epoch 1294
---------------
Epoch 1295
---------------
Epoch 1296
---------------
Epoch 1297
---------------
Epoch 1298
---------------
Epoch 1299
---------------
Epoch 1300
---------------
Epoch 1301
---------------
Epoch 1302
---------------
Epoch 1303
---------------
Epoch 1304
---------------
Epoch 1305
-

...

Epoch 18523
---------------
Epoch 18524
---------------
Epoch 18525
---------------
Epoch 18526
---------------
Epoch 18527
---------------
Epoch 18528
---------------
Epoch 18529
---------------
Epoch 18530
---------------
Epoch 18531
---------------
Epoch 18532
---------------
Epoch 18533
---------------
Epoch 18534
---------------
Epoch 18535
---------------
Epoch 18536
---------------
Epoch 18537
---------------
Epoch 18538
---------------
Epoch 18539
---------------
Epoch 18540
---------------
Epoch 18541
---------------
Epoch 18542
---------------
Epoch 18543
---------------
Epoch 18544
---------------
Epoch 18545
---------------
Epoch 18546
---------------
Epoch 18547
---------------
Epoch 18548
---------------
Epoch 18549
---------------
Epoch 18550
---------------
Epoch 18551
---------------
Epoch 18552
---------------
Epoch 18553
---------------
Epoch 18554
---------------
Epoch 18555
---------------
Epoch 18556
---------------
Epoch 18557
---------------
Epoch 18558
--------

Epoch 18823
---------------
Epoch 18824
---------------
Epoch 18825
---------------
Epoch 18826
---------------
Epoch 18827
---------------
Epoch 18828
---------------
Epoch 18829
---------------
Epoch 18830
---------------
Epoch 18831
---------------
Epoch 18832
---------------
Epoch 18833
---------------
Epoch 18834
---------------
Epoch 18835
---------------
Epoch 18836
---------------
Epoch 18837
---------------
Epoch 18838
---------------
Epoch 18839
---------------
Epoch 18840
---------------
Epoch 18841
---------------
Epoch 18842
---------------
Epoch 18843
---------------
Epoch 18844
---------------
Epoch 18845
---------------
Epoch 18846
---------------
Epoch 18847
---------------
Epoch 18848
---------------
Epoch 18849
---------------
Epoch 18850
---------------
Epoch 18851
---------------
Epoch 18852
---------------
Epoch 18853
---------------
Epoch 18854
---------------
Epoch 18855
---------------
Epoch 18856
---------------
Epoch 18857
---------------
Epoch 18858
--------

---------------
Epoch 19119
---------------
Epoch 19120
---------------
Epoch 19121
---------------
Epoch 19122
---------------
Epoch 19123
---------------
Epoch 19124
---------------
Epoch 19125
---------------
Epoch 19126
---------------
Epoch 19127
---------------
Epoch 19128
---------------
Epoch 19129
---------------
Epoch 19130
---------------
Epoch 19131
---------------
Epoch 19132
---------------
Epoch 19133
---------------
Epoch 19134
---------------
Epoch 19135
---------------
Epoch 19136
---------------
Epoch 19137
---------------
Epoch 19138
---------------
Epoch 19139
---------------
Epoch 19140
---------------
Epoch 19141
---------------
Epoch 19142
---------------
Epoch 19143
---------------
Epoch 19144
---------------
Epoch 19145
---------------
Epoch 19146
---------------
Epoch 19147
---------------
Epoch 19148
---------------
Epoch 19149
---------------
Epoch 19150
---------------
Epoch 19151
---------------
Epoch 19152
---------------
Epoch 19153
---------------
Epoc

Epoch 19415
---------------
Epoch 19416
---------------
Epoch 19417
---------------
Epoch 19418
---------------
Epoch 19419
---------------
Epoch 19420
---------------
Epoch 19421
---------------
Epoch 19422
---------------
Epoch 19423
---------------
Epoch 19424
---------------
Epoch 19425
---------------
Epoch 19426
---------------
Epoch 19427
---------------
Epoch 19428
---------------
Epoch 19429
---------------
Epoch 19430
---------------
Epoch 19431
---------------
Epoch 19432
---------------
Epoch 19433
---------------
Epoch 19434
---------------
Epoch 19435
---------------
Epoch 19436
---------------
Epoch 19437
---------------
Epoch 19438
---------------
Epoch 19439
---------------
Epoch 19440
---------------
Epoch 19441
---------------
Epoch 19442
---------------
Epoch 19443
---------------
Epoch 19444
---------------
Epoch 19445
---------------
Epoch 19446
---------------
Epoch 19447
---------------
Epoch 19448
---------------
Epoch 19449
---------------
Epoch 19450
--------

Epoch 19715
---------------
Epoch 19716
---------------
Epoch 19717
---------------
Epoch 19718
---------------
Epoch 19719
---------------
Epoch 19720
---------------
Epoch 19721
---------------
Epoch 19722
---------------
Epoch 19723
---------------
Epoch 19724
---------------
Epoch 19725
---------------
Epoch 19726
---------------
Epoch 19727
---------------
Epoch 19728
---------------
Epoch 19729
---------------
Epoch 19730
---------------
Epoch 19731
---------------
Epoch 19732
---------------
Epoch 19733
---------------
Epoch 19734
---------------
Epoch 19735
---------------
Epoch 19736
---------------
Epoch 19737
---------------
Epoch 19738
---------------
Epoch 19739
---------------
Epoch 19740
---------------
Epoch 19741
---------------
Epoch 19742
---------------
Epoch 19743
---------------
Epoch 19744
---------------
Epoch 19745
---------------
Epoch 19746
---------------
Epoch 19747
---------------
Epoch 19748
---------------
Epoch 19749
---------------
Epoch 19750
--------

In [30]:
import time
observation = env.reset()
model.eval()
done = False
a = time.time()
while done == False:
    pobservation = torch.from_numpy(observation)
    flattened_pobservation = pobservation.view(-1).float()
    policy, value = model(flattened_pobservation)
    sampler = Categorical(policy)
    action = sampler.sample()
    observation, reward, done, log = env.step(moveMapping[action.item()])
    env.render()
    
if done:
    env.close()
    
b = time.time()-a
print(b)

0.06654500961303711


In [31]:
m1 = ActorCritic()
observation = env.reset()
import time
# observation = env.reset()
m1.eval()
done = False
a = time.time()
while done == False:
    pobservation = torch.from_numpy(observation)
    flattened_pobservation = pobservation.view(-1).float()
    policy, value = m1(flattened_pobservation)
    sampler = Categorical(policy)
    action = sampler.sample()
    observation, reward, done, log = env.step(moveMapping[action.item()])
    env.render()
    
if done:
    env.close()
    
b = time.time()-a
print(b)

0.062400102615356445


In [ ]:
############################ Experimentations ###############################################

In [ ]:
# from torch.distributions import Categorical
# a = torch.tensor([20,20,20,20,20]).float()
# # Note that this is equivalent to what used to be called multinomial
# m = Categorical(a)

# li = [0,0,0,0,0]


# for i in range(100):
#     li[m.sample().item()] +=1
    
# li

In [ ]:
# a = torch.tensor(3)
# a.item()


In [ ]:
#############################################################################################

In [11]:
observation.shape

(210, 160, 3)